**Table of contents**<a id='toc0_'></a>    
- [Résumé](#toc1_)    
  - [Problématique](#toc1_1_)    
  - [Extrait](#toc1_2_)    
- [Introduction](#toc2_)    
  - [Prérequis et imports](#toc2_1_)    
  - [Fonctions utilisées](#toc2_2_)    
  - [Chargement des données](#toc2_3_)    
- [Comparatif des modèles](#toc3_)    
  - [Choix d'estimateur](#toc3_1_)    
- [Feature importance et poids des modèles](#toc4_)    
- [Hyperparamétrage](#toc5_)    
- [Influence de l'ENERGYSTAR Score](#toc6_)    
- [Comparatif avec ratio rés / non-rés](#toc7_)    
- [Conclusion](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Résumé](#toc0_)

## <a id='toc1_1_'></a>[Problématique](#toc0_)

À partir des données provenant de la ville de Seattle, nous recherchons à prédire la consommation énergétique et l'émission de CO2 des bâtiments non destinés à l'habitation, tout en évaluant l'intérêt de l' "ENERGY STAR Score" dans les prédictions d'émissions.  
Tout nouveau bâtiment aura un premier relevé de référence la première année.  

✅ MISSION 1 → réaliser une **[analyse exploratoire des données](./P3_EDA.ipynb)**  

**À partir des relevés existants et des données structurelles** des bâtiments (taille, usage, date de construction, situation géographique, ...) tenter de :  

✅ MISSION 2 → **[prédire les émissions de CO2](./P3_ML_1_GHGE.ipynb)** pour les bâtiments non mesurés et **évaluer l'intérêt de l'ENERGY STAR Score** pour la prédiction d'émissions (fastidieux à calculer, à intégrer dans la modélisation)  
👉 MISSION 3 → **prédire la consommation totale d'énergie** pour les bâtiments non mesurés  

## <a id='toc1_2_'></a>[Extrait](#toc0_)

Réutilisation du **pipeline de pré-traitement** précédent.

Le meilleur estimateur est cette fo,is la **régression linéaire simple**.

Cet estimateur n'a que **peu d'hyperparamètres** et l'hyperparamétrage **n'apporte rien au niveau des scores mesurés** (sur toutes les métriques, à savoir r², MAE et RMSE) mais permet de **forcer le modèle à se baser davantage sur les variables numériques que catégorielles**.

Les scores et analyses des poids et la feature importance montrent **à nouveau une corrélation forte avec plusieurs variables énergétique**, mais aussi avec le **quartier dans lequel le bâtiment** est situé.  
Par ordre d'importance :
1. **consommation de gaz naturel** normalisée
2. **émissions de gaz à effet de serre** normalisées
3. **consommation électrique** normalisée et **consommation de chauffage urbain** normalisée
4. dans une moindre mesure mais à souligner : le **quartier dans lequel le bâtiment est situé**

Le taux d'**usage non résidentiel des bâtiments influe sur le modèle** et est donc à définir avec les équipes métier.

L'**ENERGYSTAR Score** n'a presque aucune influence sur le modèle.

Confirmation de certains points à vérifier avec Douglas et les équipes métier :
- l'**usage non résidentiel** (équipes métier pour déterminer un seuil et améliorer le remplissage des données en amont)
- **vérifier qu'on bénéficie bien des relevés de première année** pour en déduire les variables énergétiques structurelles fortement corrélées à la cible
- **l'ENERGYSTAR Score n'a pas d'influence sur ce modèle ni sur le précédent** : il n'est d'aucun intérêt particulier pour la prédiction d'émissions comme de consommation

Enfin, contrairement à ce qu'on aurait pu imaginer intuitivement, **l'année de construction du bâtiment n'a aucun impact dans les modèles** de consommation énergétique ou d'émissions de gaz à effet de serre.

# <a id='toc2_'></a>[Introduction](#toc0_)

## <a id='toc2_1_'></a>[Prérequis et imports](#toc0_)

Pré-requis de fonctionnement :

Packages utilisés :

```
Python 3.9.18
-----
matplotlib          3.7.2
IPython             8.15.0
jupyter_client      7.4.9
jupyter_core        5.3.0
jupyterlab          3.6.3
notebook            6.5.4
numpy               1.25.2
pandas              2.0.3
plotly              5.9.0
scipy               1.11.1
seaborn             0.12.2
session_info        1.0.0
sklearn             1.3.0
```

In [1]:
import logging
import pickle
import time
from functools import partial
import warnings

import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import make_scorer

from sklearn.inspection import permutation_importance

# render in GitHub & NBViewer
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# prevent warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

# logging configuration (see all outputs, even DEBUG or INFO)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

## <a id='toc2_2_'></a>[Fonctions utilisées](#toc0_)

In [2]:
def nan_warn(df, nan_col="nan_pct", tags_col="tags", thresh=0.4):
    """
    Warns if NaNs outpass a defined threshold,
    warning showed as a tag in the dataframe on a defined column.

    Inputs:
    • df: dataframe
    • nan_col: dataframe column (string, default = "nan")
    • tags_col: dataframe column (string, default = "tags")
    • thresh: threshold for NaNs warning (float, default = 0.4)

    Output: modified dataframe

    Requirements: pandas
    """

    df_ = df.copy()
    mask = df_[nan_col] / 100 >= thresh
    df_.loc[mask, tags_col] = df_.loc[mask, tags_col] + "🚫"

    return df_

def type_tag(df, uni_col="unique", type_col="type", count_col="count",
        tags_col="tags"):
    """
    Defines a type tag of a dataframe feature,
    depending on unique values, count and dtype,
    and writes it in a tag column.

    Inputs:
    • df: dataframe
    • uni_col: dataframe column (string, default = "unique")
    • type_col: dataframe column (string, default = "type")
    • count_col: dataframe column (string, default = "type")
    • tags_col: dataframe column (string, default = "tags")

    Output: modified dataframe

    Requirements: pandas
    """

    df_ = df.copy()
    total_count = max(df_[count_col])

    # const warn
    const_mask = df_[uni_col] == 1
    df_.loc[const_mask, tags_col] = df_.loc[const_mask, tags_col] + "🔒"

    # unique warn
    uniq_mask = df_[uni_col] == total_count
    df_.loc[uniq_mask, tags_col] = df_.loc[uniq_mask, tags_col] + "💎"

    # bool = categorical feat
    is_bool_mask = df_[type_col] == "bool"
    df_.loc[is_bool_mask, tags_col] = df_.loc[is_bool_mask, tags_col] + "📦"
    
    # object categorical feat
    type_mask = df_[type_col] == "object"
    # define limit
    categ_limit = int(max(2, min(60, total_count / 1.2)))
    # filter
    categ_mask = df_[uni_col].between(2, categ_limit)
    df_.loc[(categ_mask & type_mask), tags_col] = df_.loc[
        (categ_mask & type_mask), tags_col] + "📦"

    return df_

def describe_df(df, nan_thresh=0.4):
    """
    Dataframe describer, include little more information than .describe()

    Inputs:
    • df: dataframe to be analysed
    • nan_thresh: threshold for NaNs warning (float, default = 0.4)

    Output: dataframe of data description

    Requirements: pandas, numpy
    """

    df_ = df.describe(include="all").T
    df_.sort_index(inplace=True)
    df_["unique"] = df.nunique()
    df_["type"] = df.dtypes
    df_["nan"] = df.isna().sum()
    df_["nan_pct"] = np.round(df.isna().mean()*100, 2)
    
    # tags column
    df_.insert(0, "tags", "")
    # nan warning tag
    df_ = nan_warn(df_, thresh=nan_thresh)
    # type check + const warn tag
    df_ = type_tag(df_,
        uni_col="unique",
        type_col="type",
        count_col="count",
        tags_col="tags",
        )
    
    df_ = df_.fillna("-")

    return df_

def impact_classif(value, thresh=30):
    """
    Returns an impact classification depending on a value
    and a threshold.

    Positional arguments: 
    -------------------------------------
    value: float or int: between 0 and 100

    Optional arguments: 
    -------------------------------------
    thresh: float or int: threshold to adjust the function, default=30

    Output: string, warning intensity (int)
    """

    if value == 0:
        return "⌀", False
    elif 0 < value < (thresh / 6):
        return "--", False
    elif (thresh / 6) <= value < (thresh / 3):
        return "-", False
    elif (thresh / 3) <= value < (thresh * 2 / 3):
        return "+", False
    elif (thresh * 2 / 3) <= value < thresh:
        return "++", False
    elif thresh <= value < (thresh + (thresh / 3)):
        return "⚠️", 1
    elif (thresh + (thresh / 3)) <= value < (thresh + (2 * thresh / 3)):
        return "⚠️⚠️", 2
    elif (thresh + (2 * thresh / 3)) <= value < 75:
        return "⚠️⚠️⚠️", 3
    elif value >= 75:
        return "☠️", 4
    else:
        return "❓", False

def impact(df_before, df_after, monitored=None):
    """
    Returns an impact dataframe from an original and
    a second dataframe.
    Impact is calculated on the columns and population,
    plus on some optional arguments.

    Positional arguments: 
    -------------------------------------
    df_before: dataframe: original dataframe (starting point, before action)
    df_before: dataframe: original dataframe (starting point, after action)

    Optional arguments: 
    -------------------------------------
    monitored: list of strings: Columns to check.
               ⚠️ Columns must be present in both dataframes.
               Default = None

    Output: dataframe in logging.info()

    Required modules: pandas, numpy, logging
    """

    pop_bef = df_before.shape[0]
    cols_bef = df_before.shape[1]
    
    pop_aft = df_after.shape[0]
    cols_aft = df_after.shape[1]

    diff_pop = pop_bef - pop_aft
    diff_cols = cols_bef - cols_aft

    prct_pop_num = np.round(diff_pop / pop_bef * 100, 2)
    prct_cols_num = np.round(diff_cols / cols_bef * 100, 2)
    prct_pop = f"{prct_pop_num}%"
    prct_cols = f"{prct_cols_num}%"

    imp_cols = impact_classif(prct_cols_num)
    imp_pop = impact_classif(prct_pop_num)

    # list of potential warnings
    warn_logs = [imp_cols[1], imp_pop[1]]

    _df_ = pd.DataFrame([
        [cols_bef, pop_bef],
        [cols_aft, pop_aft],
        [diff_cols, diff_pop],
        [prct_cols, prct_pop],
        [imp_cols[0], imp_pop[0]]],
        columns=["Columns", "Population"],
        index=["Before", "After", "Difference", "Prct", "IMPACT"]
        )

    if monitored:
        for m in monitored:
            # get output from command
            before_ = df_before[m].count()
            after_ = df_after[m].count()
            prct_ = np.round((before_ - after_) / before_ * 100, 2)
            imp = impact_classif(prct_)
            # add in DF
            _df_[m] = [before_,
                    after_,
                    before_ - after_,
                    f"{prct_}%",
                    imp[0]
                ]
            # add potential warning
            warn_logs.append(imp[1])

    if (1 in warn_logs or 2 in warn_logs or 3 in warn_logs):
        return logging.warning(display(_df_))
    elif 4 in warn_logs:
        return logging.critical(display(_df_))
    else:
        return logging.info(display(_df_))

def data_cleaner(df, min_nr_pct=0.0001, verbose=False):
    """

    Data cleaning, grouping explicitly all previously used cleaning methods.

    Inputs:
    • df: dataframe to clean
    • min_nr_pct: minimal non residential percentage (float, default = 0.0001)
    • verbose: determines whether logs are shown or not (bool, default = False)

    Output: cleansed dataframe

    Requirements: numpy, pandas, logging
    """

    # DROP DUPLICATES
    # *************************************************************************
    df_ = df.drop_duplicates()

    # KEEP COMPLIANT DATA ONLY
    # *************************************************************************
    df_ = df_.loc[df_["ComplianceStatus"] == "Compliant"]

    # CLEAN NEIGHBORHOOD
    # *************************************************************************
    # case harmonization
    df_["Neighborhood"] = df_["Neighborhood"].str.upper()
    # duplicate deletion
    df_.loc[df_["Neighborhood"] == "DELRIDGE NEIGHBORHOODS",
        "Neighborhood"] = "DELRIDGE"
    
    # DELETE USELESS FEATURES
    # *************************************************************************
    df_.drop([
        "Address",
        "City",
        "Comments",
        "ComplianceStatus",
        "CouncilDistrictCode",
        "DataYear",
        "DefaultData",
        # 'Electricity(kBtu)',
        "Electricity(kWh)",
        # "ENERGYSTARScore",
        # 'GHGEmissionsIntensity',
        "Latitude",
        "ListOfAllPropertyUseTypes",
        "Longitude",
        # 'NaturalGas(kBtu)',
        "NaturalGas(therms)",
        # "Neighborhood",
        # "NumberofBuildings",
        # "NumberofFloors",
        "OSEBuildingID",
        "Outlier",
        # 'PropertyGFABuilding(s)',
        # 'PropertyGFAParking',
        # 'PropertyGFATotal',
        "PropertyName",
        "SiteEnergyUse(kBtu)",
        "SiteEnergyUseWN(kBtu)",
        "SiteEUI(kBtu/sf)",
        # 'SiteEUIWN(kBtu/sf)',
        "SourceEUI(kBtu/sf)",
        "SourceEUIWN(kBtu/sf)",
        "State",
        # 'SteamUse(kBtu)',
        "TaxParcelIdentificationNumber",
        "TotalGHGEmissions",
        # 'YearBuilt',
        "YearsENERGYSTARCertified",
        "ZipCode",
        ], axis=1, inplace=True)
    
    # BUILDING RATIO
    # *************************************************************************
    df_["BuildingRatio"] = df_["PropertyGFABuilding(s)"]\
        / df_["PropertyGFATotal"]
    
    # PARKING RATIO
    # *************************************************************************
    df_["pkg_gfa"] = 0
    # seek parking GFA and add up
    cols = ["ThirdLargestPropertyUseType", "SecondLargestPropertyUseType",
        "LargestPropertyUseType"]
    for c in cols:
        gfa = c + "GFA"
        is_pkg = df_[c].str.lower().str.contains(r'parking', na=False)
        df_[gfa].where(is_pkg, 0, inplace=True)
        # add GFA to total
        df_["pkg_gfa"] += df_[gfa]
    # keep highest GFA
    df_.loc[df_["pkg_gfa"] > df_["PropertyGFAParking"],
        "PropertyGFAParking"] = df_["pkg_gfa"]
    # apply % on total GFA
    df_["ParkingRatio"] = df_["PropertyGFAParking"] / df_["PropertyGFATotal"]
    df_.drop(["pkg_gfa"], axis=1, inplace=True)

    # NON-RESIDENTIAL RATIO (ALL USAGE FEATS COMPILATION)
    # *************************************************************************
    df_["non_res_gfa"] = 0
    lput_notna = df_["LargestPropertyUseType"] != np.NaN
    cols = ["ThirdLargestPropertyUseType", "SecondLargestPropertyUseType",
        "LargestPropertyUseType"
    ]
    # LargestPropertyUseType != NaN
    for c in cols:
        gfa = c + "GFA"
        is_res = df_[c].str.lower().str.contains(
            r'(?<!(non))(residential|multifamily|residence)', na=True)
        df_[gfa].mask(lput_notna & is_res, 0, inplace=True)
        # add GFA to total
        df_["non_res_gfa"] += df_[gfa]
    # LargestPropertyUseType == NaN
    zero_non_res_gfa = df_["non_res_gfa"] != 0
    is_res = df_["PrimaryPropertyType"].str.lower().str.contains(
        r'(?<!(non))(residential|multifamily|residence)', na=True)
    df_["non_res_gfa"].where(lput_notna & is_res | zero_non_res_gfa,
        df_["PropertyGFATotal"], inplace=True)
    # apply % on total GFA
    df_["NonResidentialRatio"] = df_["non_res_gfa"] / df_["PropertyGFATotal"]
    df_.loc[df_["NonResidentialRatio"] > 1, "NonResidentialRatio"] = 1
    df_.drop(["non_res_gfa"], axis=1, inplace=True)

    # DROP POPULATION UNDER A NON-RESIDENTIAL RATIO
    # *************************************************************************
    df_ = df_.loc[(df_["NonResidentialRatio"] >= min_nr_pct)]

    # CHANGE RAW ENERGY VALUES TO ENERGY INTENSITY
    # *************************************************************************
    df_["SteamUse_I(kBtu/sf)"] = df_["SteamUse(kBtu)"]\
        / df_["PropertyGFATotal"]
    df_["Electricity_I(kBtu/sf)"] = df_["Electricity(kBtu)"]\
        / df_["PropertyGFATotal"]
    df_["NaturalGas_I(kBtu/sf)"] = df_["NaturalGas(kBtu)"]\
        / df_["PropertyGFATotal"]
        
    # SET MINIMUM NUMBER OF FLOORS TO 1 AND ADD AREA PER FLOOR FEATURE
    # *************************************************************************
    df_.loc[(df_["NumberofFloors"] == 0) & (df_["BuildingRatio"] > 0),
        "NumberofFloors"] = 1
    df_["AreaPerFloor(sf)"] = df_["NumberofFloors"]\
        / df_["PropertyGFATotal"]

    # SET MINIMUM NUMBER OF BUILDINGS TO 1 AND ADD AREA PER BUILDING FEATURE
    # *************************************************************************
    df_.loc[(df_["NumberofBuildings"] == 0) & (df_["BuildingRatio"] > 0),
        "NumberofBuildings"] = 1
    df_["AreaPerBldg(sf)"] = df_["NumberofBuildings"]\
        / df_["PropertyGFATotal"]

    # CHASE STATISTICAL OUTLIERS
    # *************************************************************************
    df_ = df_.loc[df_["GHGEmissionsIntensity"] <= 20] # 2 individuals
    df_ = df_.loc[df_["AreaPerFloor(sf)"] <= 0.004] # 1 individual
    df_ = df_.loc[df_["BuildingRatio"] >= 0.4] # 19 individuals
    df_ = df_.loc[df_["NumberofBuildings"] <= 20] # 3 individuals
    df_ = df_.loc[df_["NumberofFloors"] <= 40] # 16 individuals
    df_ = df_.loc[df_["ParkingRatio"] <= 0.8] # 8 individuals
    df_ = df_.loc[df_["PropertyGFATotal"] <= 2000000] # 1 individual
    df_ = df_.loc[df_["SteamUse_I(kBtu/sf)"] <= 100] # 6 individuals
    df_ = df_.loc[df_["Electricity_I(kBtu/sf)"] <= 350] # 6 individuals

    # DELETE LAST USELESS FEATURES
    # *************************************************************************
    df_.drop([
        "BuildingType",
        "PrimaryPropertyType",
        "LargestPropertyUseType",
        "LargestPropertyUseTypeGFA",
        "SecondLargestPropertyUseType",
        "SecondLargestPropertyUseTypeGFA",
        "ThirdLargestPropertyUseType",
        "ThirdLargestPropertyUseTypeGFA",
        "PropertyGFABuilding(s)", # => BuildingRatio
        "PropertyGFAParking", # => ParkingRatio
        "SteamUse(kBtu)",
        "Electricity(kBtu)",
        "NaturalGas(kBtu)",
        "NumberofFloors",
        "NumberofBuildings",
    ], axis=1, inplace=True)

    # DROP NANS EXCEPT FOR ENERGY STAR SCORE FEATURE
    # *************************************************************************
    feats = ["ENERGYSTARScore"]
    df_.dropna(subset=df_.columns.difference(feats), inplace=True)

    # SORT COLUMNS
    # *************************************************************************
    df_.sort_index(axis=1, inplace=True)

    # SHOW GLOBAL IMPACT
    if verbose:
        logging.info("""\n***************************************************
    👇👇   GLOBAL IMPACT  👇👇""")
        impact(df, df_)

    return df_

def select_model(df, non_res_min_usage, target, random_state, verbose=False,
        plot=True):
    """
    Cross validation model selection.
    
    Inputs:
    • df: dataframe to process
    • non_res_min_usage: minimal non residential percentage (float)
    • target: targetted feature, must be in df (string)
    • random_state: random state for kfold splits
    • verbose: determines whether logs are shown or not (bool, default = False)
    • plot: plots results or not (bool, default = False)

    Output:
    • dict containing :
        • df, X and y: 3 dataframes (cleansed df, X, y)
        • pipes: model pipelines
        • scores: all scores

    Requirements: numpy, pandas, sklearn, logging, plotly
    """

    df_ = df.copy()
    output = {}

    # PARAMS
    # *************************************************************************
    classif_cols = ["Neighborhood"]

    # CLEAN
    # *************************************************************************
    df_ = data_cleaner(df_, non_res_min_usage)

    # REMAINING PARAMS (must be done after cleaning)
    # *************************************************************************
    num_cols = df_.drop(classif_cols, axis=1).columns.to_list()
    num_cols.remove(target)

    # PIPELINES DEFINITIONS
    # *************************************************************************
    col_transf = ColumnTransformer(
        [
            ('one_hot', OneHotEncoder(handle_unknown='ignore'), classif_cols),
            ('min_max_scaler', MinMaxScaler(), num_cols),
        ],
        remainder = 'passthrough',
        verbose_feature_names_out=False
    )

    imputer = Pipeline(
        [
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ]
    )

    preprocessor = Pipeline(
        [
            ("col_transf", col_transf),
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ]
    )

    # ESTIMATORS
    # *************************************************************************
    models_names = ["Dummy", "LinReg", "SVR", "RdmForestReg", "GradBoostReg"]
    
    # pipelines
    pipelines = [
        Pipeline([('Dummy', DummyRegressor(strategy="mean"))]),
        Pipeline([('LinReg', LinearRegression())]),
        Pipeline([('SVR', svm.SVR(kernel='linear'))]),
        Pipeline([('RdmForestReg', RandomForestRegressor())]),
        Pipeline([('GradBoostReg', GradientBoostingRegressor())]),
    ]

    # PREPROCESSING & X,Y SPLIT
    # *************************************************************************
    df_ = pd.DataFrame(preprocessor.fit_transform(df_),
        columns=preprocessor[0:].get_feature_names_out())

    # X, y split
    y = df_[target]
    X = df_.drop(target, axis=1)

    # output data
    output["df"] = df_
    output["X"] = X
    output["y"] = y

    # CROSS-VALIDATION
    # *************************************************************************
    # for output
    output["pipes"] = {}
    scores = pd.DataFrame(columns=["estimator", "fold", "r2", "mae", "rmse",
        "fit_time", "mean_r2", "mean_mae", "mean_rmse", "mean_fit_time"])
    
    # loop over estimators
    for p, name in zip(pipelines, models_names):

        scoring = ["r2", "neg_mean_absolute_error",
            "neg_root_mean_squared_error"]

        # 5-fold CV with randomization
        kf = KFold(n_splits=5, random_state=random_state, shuffle=True)
        cv = cross_validate(p, X, y, cv=kf, n_jobs=-1, scoring=scoring)

        # output
        output["pipes"][name] = p

        # loop over CV folds
        for i,s in enumerate(cv["test_r2"]):
            scores.loc[len(scores)] = [
                name,
                i+1,
                cv["test_r2"][i],
                -cv["test_neg_mean_absolute_error"][i],
                -cv["test_neg_root_mean_squared_error"][i],
                cv["fit_time"][i],
                0.0,
                0.0,
                0.0,
                0.0,
                ]

        # create means
        scores.loc[scores["estimator"] == name, "mean_r2"] = scores.loc[
            scores["estimator"] == name, "r2"].mean()
        scores.loc[scores["estimator"] == name, "mean_mae"] = scores.loc[
            scores["estimator"] == name, "mae"].mean()
        scores.loc[scores["estimator"] == name, "mean_rmse"] = scores.loc[
            scores["estimator"] == name, "rmse"].mean()
        scores.loc[scores["estimator"] == name, "mean_fit_time"] = scores.loc[
            scores["estimator"] == name, "fit_time"].mean()

    output["scores"] = scores

    # # plot
    if plot:
        # for m in ["r2", "mae", "rmse", "fit_time"]:
        # display only r2
        for m in ["r2"]:
            # plot it
            fig = go.Figure()
            x_ = scores["fold"].unique()
            fig.add_trace(go.Scatter(
                x=x_,
                y=scores.loc[scores["estimator"] == "LinReg", m],
                name="LinReg",
                line=dict(color='blue'),
                ))
            fig.add_trace(go.Scatter(
                x=x_,
                y=scores.loc[scores["estimator"] == "SVR", m],
                name="SVR",
                line=dict(color='red'),
                ))
            fig.add_trace(go.Scatter(
                x=x_,
                y=scores.loc[scores["estimator"] == "RdmForestReg", m],
                name="RdmForestReg",
                line=dict(color='green'),
                ))
            fig.add_trace(go.Scatter(
                x=x_,
                y=scores.loc[scores["estimator"] == "GradBoostReg", m],
                name="GradBoostReg",
                line=dict(color='goldenrod'),
                ))
            
            # plot means
            fig.add_trace(go.Scatter(
                x=x_,
                y=scores.loc[scores["estimator"] == "LinReg",
                    str("mean_" + m)],
                name="LinReg_mean",
                line=dict(color='blue', dash="dot"),
                opacity=0.5,
                ))
            fig.add_trace(go.Scatter(
                x=x_,
                y=scores.loc[scores["estimator"] == "SVR",
                    str("mean_" + m)],
                name="SVR_mean",
                line=dict(color='red', dash="dot"),
                opacity=0.5,
                ))
            fig.add_trace(go.Scatter(
                x=x_,
                y=scores.loc[scores["estimator"] == "RdmForestReg",
                    str("mean_" + m)],
                name="RdmForestReg_mean",
                line=dict(color='green', dash="dot"),
                opacity=0.5,
                ))
            fig.add_trace(go.Scatter(
                x=x_,
                y=scores.loc[scores["estimator"] == "GradBoostReg",
                    str("mean_" + m)],
                name="GradBoostReg_mean",
                line=dict(color='goldenrod', dash="dot"),
                opacity=0.5,
                ))

            fig.update_layout(width=700, height=400, title=f"{m} overview")
            fig.show()
    
    if verbose:
        display(scores)

    return output

def train_model(df, non_res_min_usage, target, random_state, fi=True,
        ESS=True, verbose=False):
    """
    Processes global pipeline and returns a fitted model.
    
    Inputs:
    • df: dataframe to process
    • non_res_min_usage: minimal non residential percentage (float)
    • target: targetted feature, must be in df (string)
    • random_state: random state for kfold splits
    • fi: fit_intercept hyperparameter on model (bool, default = True)
    • ESS: if False, drops the ENERGYSTARScore feature (bool, default = True)
    • verbose: determines whether logs are shown or not (bool, default = False)

    Output:
    Dict within:
        • splits' dataframes ("X_train", "X_test", "y_train", "y_test")
        • fitted model ("model")
        • scores on test set ("r2", "mae", "rmse", "fit_time")

    Requirements: numpy, pandas, sklearn
    """

    df_ = df.copy()
    output = {}

    # PARAMS
    # *************************************************************************
    classif_cols = ["Neighborhood"]

    # CLEAN
    # *************************************************************************
    df_ = data_cleaner(df_, non_res_min_usage)
    # drop ENERGYSTARScore if asked
    if not ESS:
        df_.drop("ENERGYSTARScore", axis=1, inplace=True)

    # REMAINING PARAMS (must be done after cleaning)
    # *************************************************************************
    num_cols = df_.drop(classif_cols, axis=1).columns.to_list()
    num_cols.remove(target)

    # PIPELINES DEFINITIONS
    # *************************************************************************
    col_transf = ColumnTransformer(
        [
            ('one_hot', OneHotEncoder(handle_unknown='ignore'), classif_cols),
            ('min_max_scaler', MinMaxScaler(), num_cols),
        ],
        remainder = 'passthrough',
        verbose_feature_names_out=False
    )

    imputer = Pipeline(
        [
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ]
    )

    preprocessor = Pipeline(
        [
            ("col_transf", col_transf),
            ('knn_imputer', KNNImputer(n_neighbors=5)),
        ]
    )


    # PREPROCESSING & X,Y SPLIT
    # *************************************************************************
    df_ = pd.DataFrame(preprocessor.fit_transform(df_),
        columns=preprocessor[0:].get_feature_names_out())

    # X, y + train / test split
    # *************************************************************************
    y = df_[target]
    X = df_.drop(target, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
        random_state=random_state)

    # output data
    output["X_train"] = X_train
    output["X_test"] = X_test
    output["y_train"] = y_train
    output["y_test"] = y_test

    # FIT
    # *************************************************************************
    linreg = Pipeline([('LinReg', LinearRegression(fit_intercept=fi))])
    start = time.time()
    fitted = linreg.fit(X_train, y_train)
    end = time.time()

    fit_time = end - start

    r2 = round(r2_score(y_test, fitted.predict(X_test)), 3)
    mae = round(mean_absolute_error(y_test, fitted.predict(X_test)), 3)
    rmse = round(mean_squared_error(y_test, fitted.predict(X_test),
        squared = False), 3)

    # output model and scores
    output["fitted_model"] = fitted
    output["r2"] = r2
    output["mae"] = mae
    output["rmse"] = rmse
    output["fit_time"] = fit_time

    return output

def permut_fi(model, X, y):
    """
    Returns and plots feature importance.
    """

    fig = go.Figure()
    fi = permutation_importance(model, X, y, scoring='neg_mean_squared_error')
    importance = fi.importances_mean

    # plot it
    fig.add_trace(go.Bar(x=X.columns, y=importance, name="LinReg"))

    # display adjustments
    fig.update_layout(title="Feature Importance",
        width=800, height=500)

    fig.update_traces(textposition="outside", cliponaxis=False)
    fig.update_xaxes(tickangle=60)

    # add vertical lines
    for col in range(1, X.shape[1]):
        fig.add_vline(x=col+0.5, line_width=1,
            line_dash="dash", line_color="grey")

    fig.show()

    return fi


## <a id='toc2_3_'></a>[Chargement des données](#toc0_)

In [3]:
DATASETS_PATH = "./"
dataset_name = "oc_p3_2016_Building_Energy_Benchmarking.csv"
data_raw = pd.read_csv(DATASETS_PATH+dataset_name)

# <a id='toc3_'></a>[Comparatif des modèles](#toc0_)

Comparaison des modèles via une validation croisée à 5 blocs avec un aléa fixé par un random state.  

In [4]:
non_res_min_usage = 0.0001
target = "SiteEUIWN(kBtu/sf)"

Tests sur plusieurs random states :

In [5]:
rs = 0
model_compare = select_model(data_raw, non_res_min_usage, target, rs, plot=True)

In [6]:
rs = 1
model_compare = select_model(data_raw, non_res_min_usage, target, rs, plot=True)

In [7]:
rs = 2
model_compare = select_model(data_raw, non_res_min_usage, target, rs, plot=True)

In [8]:
rs = 3
model_compare = select_model(data_raw, non_res_min_usage, target, rs, plot=True)

In [9]:
rs = 9
model_compare = select_model(data_raw, non_res_min_usage, target, rs, plot=True)

In [10]:
rs = 42
model_compare = select_model(data_raw, non_res_min_usage, target, rs, plot=True)

## <a id='toc3_1_'></a>[Choix d'estimateur](#toc0_)

Sur cette cible, c'est **la régression linéaire simple qui domine** les autres estimateurs avec à la fois un r² plus élevé, une RMSE plus basse et un temps d'entraînement moins long.

On constate cependant des **scores très élevés**.  
Pour aller plus loin et sachant la situation quand à la forte corrélation des données énergétiques précédentes, il est intéressant de **confirmer les résultats avec une feature importance et l'analyse du poids des modèles**.

# <a id='toc4_'></a>[Feature importance et poids des modèles](#toc0_)

La feature importance varie légèrement selon l'échantillonnage, avec parfois une priorité sur les variables de classification, comme c'est le cas sur le random state 42 :

In [11]:
rs=42
linreg = train_model(data_raw, non_res_min_usage, target, rs)

print(f'Linear regression r² score on test split: r² = {linreg["r2"]}, '\
    + f'mae = {linreg["mae"]}, rmse = {linreg["rmse"]}')
    
pfi = permut_fi(linreg["fitted_model"], linreg["X_train"], linreg["y_train"])

Linear regression r² score on test split: r² = 0.92, mae = 9.246, rmse = 16.595


Pour harmoniser ceci, on utilise l'hyparamètre `fit_intercept` détaillé ci-après pour **utiliser plutôt les variables numériques**, comme c'est le cas dans la majeure partie des échantillons.  
Notons que malgré le changement, **les scores de nos métriques restent semblables**, donc la qualité des prédictions également.

In [12]:
rs=42
linreg_nofi = train_model(data_raw, non_res_min_usage, target, rs, fi=False)

print(f'Linear regression r² score on test split: r² = {linreg_nofi["r2"]}, '\
    + f'mae = {linreg_nofi["mae"]}, rmse = {linreg_nofi["rmse"]}')
pfi = permut_fi(linreg_nofi["fitted_model"], linreg_nofi["X_train"],
    linreg_nofi["y_train"])

Linear regression r² score on test split: r² = 0.92, mae = 9.258, rmse = 16.592


On peut comparer les poids de chaque modèle variable par variable :

In [13]:
coeffs = pd.DataFrame(columns = linreg["X_train"].columns)
coeffs.loc["LinReg_coef_intercept"] = linreg["fitted_model"][0].coef_
coeffs.loc["LinReg_coef_no_intercept"] = linreg_nofi["fitted_model"][0].coef_

display(coeffs.T)

,LinReg_coef_intercept,LinReg_coef_no_intercept
Neighborhood_BALLARD,-2.528975e+14,55.528434
Neighborhood_CENTRAL,-2.528975e+14,56.250912
Neighborhood_DELRIDGE,-2.528975e+14,60.330998
Neighborhood_DOWNTOWN,-2.528975e+14,54.172023
Neighborhood_EAST,-2.528975e+14,55.374072
Neighborhood_GREATER DUWAMISH,-2.528975e+14,53.348806
Neighborhood_LAKE UNION,-2.528975e+14,57.299460
Neighborhood_MAGNOLIA / QUEEN ANNE,-2.528975e+14,54.023376
Neighborhood_NORTH,-2.528975e+14,56.683895
Neighborhood_NORTHEAST,-2.528975e+14,54.842092


On constate donc que la consommation énergétique normalisée du site est dépendante :

1. de la **consommation de gaz naturel** normalisée
2. des **émissions de gaz à effet de serre** normalisées
3. à la fois de la **consommation électrique** normalisée et de la **consommation de chauffage urbain** normalisée
4. dans une moindre mesure mais tout de même visible sur la feature importance comme sur la répartition des poids : le **quartier dans lequel le bâtiment est situé**

# <a id='toc5_'></a>[Hyperparamétrage](#toc0_)

Vu plus haut, l'hyparamètre `fit_intercept` est le seul de cet estimateur à être potentiellement utile ici.  

L'*intercept* est l'intersection entre l'axe des ordonnées et la droite que le modèle de régression linéaire essaye de trouver.  
Il correspond à la valeur de $y$ pour $x = 0$ dans l'équation $y = ax + b$ ($y$ est la variable cible, $x$ la variable d'entrée, $a$ la pente de la droite et $b$ l'intercept).

C'est un booléen qui, selon sa valeur, contrôle ou non l'intercept doit être ajusté lors de l'entraînement du modèle de régression linéaire :
- si actif (par défaut), le modèle ajustera le meilleur intercept pour minimiser l'erreur de prédiction
- si inactif (`fit_intercept=False`), le modèle n'ajustera d'intercept et la régression passera toujours par l'origine et la droite n'aura pas de terme constant $b$.

Comme vu ci-avant, l'hyperparamétrage agit sur la feature importance et la répartition des poids, cependant il n'a **pas d'influence sur les scores renvoyés par les métriques utilisées**, à savoir r², MAE et RMSE.

**Son ajustement n'améliore ni ne détériore pas le modèle** dans notre cas.

# <a id='toc6_'></a>[Influence de l'ENERGYSTAR Score](#toc0_)

Lors de la feature importance et de l'analyse du poids des modèles ci-avant, il **avait été constaté que l'influence de la variable "ENERGYSTARScore" était presque nulle**.  

L'impact de sa présence ou non peut être monitoré par l'**entraînement de notre modèle sur 10 échantillons aléatoires avec et sans la présence de cette variable** dans le jeu de données :

In [14]:
nrmu = 0.0001
target = "SiteEUIWN(kBtu/sf)"
rdm_states = [*range(10)]

scores = pd.DataFrame(columns=["rdm_st", "ESS", "r2", "mae", "rmse"])

for rs in rdm_states:
    # with ENERGYSTARScore
    m_ess = train_model(data_raw, nrmu, target, rs, ESS=True)
    # without ENERGYSTARScore
    m_no_ess = train_model(data_raw, nrmu, target, rs, ESS=False)
    # save in scores DF
    scores.loc[len(scores)] = [rs, 1,
        m_ess["r2"], m_ess["mae"], m_ess["rmse"]]
    scores.loc[len(scores)] = [rs, 0,
        m_no_ess["r2"], m_no_ess["mae"], m_no_ess["rmse"]]

In [15]:
# plot results
fig = make_subplots(rows=3, cols=1)

x_ = rdm_states
colors = ['#3887ba', '#b1382c']

# loop over metrics
for i, e in enumerate(["r2", "mae", "rmse"]):
    # define scores
    y_ess = scores.loc[scores["ESS"] == 1, e]
    y_no_ess = scores.loc[scores["ESS"] == 0, e]
    
    # define means
    mean_ess = [scores.loc[scores["ESS"] == 1, e].mean()]*10
    mean_no_ess = [scores.loc[scores["ESS"] == 0, e].mean()]*10

    # plot metric for ESS and no ESS
    fig.add_trace(go.Scatter(x=x_, y=y_ess, name=f"{e}_ESS",
        legendgroup=f"group{i}", line=dict(color=colors[0])), row=i+1, col=1)
    fig.add_trace(go.Scatter(x=x_, y=y_no_ess, name=f"{e}_no_ESS",
        legendgroup=f"group{i}", line=dict(color=colors[1])), row=i+1, col=1)

    # add means for ESS and no ESS
    fig.add_trace(go.Scatter(x=x_, y=mean_ess, name=f"{e}_mean_ESS",
        legendgroup=f"group{i}", line=dict(dash="dot", color=colors[0]),
        opacity=0.5), row=i+1, col=1)
    fig.add_trace(go.Scatter(x=x_, y=mean_no_ess, name=f"{e}_mean_no_ESS",
        legendgroup=f"group{i}", line=dict(dash="dot", color=colors[1]),
        opacity=0.5), row=i+1, col=1)

# display adjustments
fig.update_layout(title="ENERGYSTARScore influence on score", width=900)
# display x-axis values for last axis only
for r in range(1, i+1):
    fig.update_xaxes(title_text='', tickvals=[], row=r, col=1)

fig.show()

Les changements apportés par la présence ou l'absence del'ENERGYSTARScore sont tout à fait mineurs : **l'ENERGYSTAR Score a une influence négligeable sur ce modèle**.

# <a id='toc7_'></a>[Comparatif avec ratio rés / non-rés](#toc0_)

Comme vu précédemment, le ratio d'utilisation résidentielle et non résidentielle permet un **ajustement du modèle par les professionnels du domaine**.  
Afin de le comparer, le même paramétrage sera utilisé, à savoir **une utilisation non résidentielle minimale entre 0.01% et 100%**, sur une échelle logarithmique et pour 10 random states différents :

In [16]:
target = "SiteEUIWN(kBtu/sf)"
rdm_states = [*range(10)]
nrmus = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 0.75, 1]

# prepare scores means lists
r2_means = [] ; mae_means = [] ; rmse_means = []

# loop over non residential minimum usages
for u in nrmus:
    r2 = [] ; mae = [] ; rmse = []
    # loop over random states
    for rs in rdm_states:
        # run process
        m = train_model(data_raw, u, target, rs)
        # add to scores temp lists
        r2.append(m["r2"])
        mae.append(m["mae"])
        rmse.append(m["rmse"])

    # to scores means lists
    r2_means.append(np.mean(r2))
    mae_means.append(np.mean(mae))
    rmse_means.append(np.mean(rmse))

# get best param
best_r2 = max(r2_means)
best_nrmu = nrmus[r2_means.index(best_r2)]

# plot it
fig = go.Figure()
x_ = nrmus
fig.add_trace(go.Scatter(x=x_, y=r2_means, name="r²"))
fig.add_trace(go.Scatter(x=x_, y=mae_means, name="MAE"))
fig.add_trace(go.Scatter(x=x_, y=rmse_means, name="RMSE"))
fig.update_xaxes(type="log")
fig.update_layout(width=600, height=400,
    title=f"SVR best param: non_res={best_nrmu} (r²={best_r2 :.3f})")
fig.show()

Que ce soit pour le r², la MAE ou la RMSE, il y a une différence marquée à partir de 0.1% d'usage non résidentiel et le score évolue par la suite.  
**Le pourcentage d'usage non résidentiel d'un bâtiment a donc un impact sur le modèle**.

Il est donc d'autant plus important de faire le point avec les équipes métier pour faire un choix sur cette variable.

# <a id='toc8_'></a>[Conclusion](#toc0_)

Cette fois-ci, c'est l'**estimateur de régression linéaire simple qui fonctionne le plus efficacement**, avec des scores et analyses des poids et feature importance qui montrent là encore une **corrélation forte avec plusieurs variables**.  

Attention néanmoins à prendre une décision concernant le taux d'**usage non résidentiel des bâtiments qui influe sur le modèle**.

L'**ENERGYSTAR Score** n'a cependant presque aucune influence sur le modèle.

On constate également une **très forte corrélation entre la cible (consommation énergétique normalisée du site) et plusieurs variables énergétiques structurelles** : 
1. **consommation de gaz naturel** normalisée
2. **émissions de gaz à effet de serre** normalisées
3. **consommation électrique** normalisée et **consommation de chauffage urbain** normalisée
4. dans une moindre mesure mais à souligner : le **quartier dans lequel le bâtiment est situé**

Cela confirme certains points à vérifier avec Douglas et les équipes métier et déjà abordés dans l'étude précédente, concernant :
- l'**usage non résidentiel** (équipes métier pour déterminer un seuil et améliorer le remplissage des données en amont)
- **vérifier qu'on bénéficie bien des relevés de première année** pour en déduire les variables énergétiques structurelles fortement corrélées à la cible
- **l'ENERGYSTAR Score n'a pas d'influence sur ce modèle ni sur le précédent** : il n'est d'aucun intérêt particulier pour la prédiction d'émissions comme de consommation

Aussi, contrairement à ce qu'on aurait pu imaginer intuitivement, **l'année de construction du bâtiment n'a aucun impact dans les modèles** de consommation énergétique ou d'émissions de gaz à effet de serre.